# 1. Find the address of your FLM in your LAN

In [ ]:
from zeroconf_discover import discover_flm

In [ ]:
flm = discover_flm()

In [ ]:
flm

# 2. Get your Sensor Configurations

In [ ]:
from mqtt_discover import discover_config

In [ ]:
config = discover_config(flm)

In [ ]:
config

# 3. Create a handler per sensor

In [ ]:
from handlers import SensorHandler

## 3.1 Create a configuration dict

You can just copy and paste from the big config. But think about gauges and counters!

In [ ]:
sensor_config = {'data_type': 'gauge',
  'enable': 1,
  'id': '064fe87f3b0879ae531fa71dee4ac037',
  'port': [1],
  'rid': 0,
  'subtype': 'pplus',
  'tmpo': 1,
  'type': 'electricity'}

## 3.2 Define a condition function

The condition function is called every time a message arrives on the topic for the sensor. It should return TRUE or FALSE

It receives the arguments `value` (float), `timestamp` (int), `unit` (str) and `kwargs`. (kwargs is a dict that contains the sensor config)

In [ ]:
# in this example I'm only doing something with the value
# but make sure you include **kwargs to catch any argument you don't need
def condition(value, **kwargs): 
    return value > 200

## 3.3 Define a callback function

The callback function is called every time the condition function evaluates to True.

You can define multiple callbacks and wrap them in a list.

In [ ]:
def callback(value, timestamp, unit, **kwargs):
    print("Value at {}: {} {}!".format(timestamp, value, unit))

## 3.4 Bind everything together

In [ ]:
handler = SensorHandler(
    sconfig=sensor_config,
    condition=condition,
    callbacks=[callback]
)

## 3.5 Test your functions

In [ ]:
# should have no effect
handler.eval_condition(value=100, timestamp=0, unit='Wh')

In [ ]:
# should trigger the callback
handler.eval_condition(timestamp=0,value=300, unit='Wh')

# 4. Create an MQTT subscriber, and run!

In [ ]:
from mqtt_subscriber import Subscriber

In [ ]:
subscriber = Subscriber(flm=flm, handlers=[handler])

In [ ]:
subscriber.run()